In [ ]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pickle
with open('grafo_bipartito.pickle', 'rb') as file:
    G = pickle.load(file)

In [ ]:
from collections import defaultdict

# Creare un dizionario per conteggiare il numero di archi uscenti da ciascun nodo film
movie_out_degree_count = defaultdict(int)

# Iterare attraverso tutti i nodi nella seconda bipartizione (nodi film)
for node in G.nodes():
    if G.nodes[node]['bipartite'] == 1:
        movie_out_degree_count[node] = len(list(G.neighbors(node)))

# Conta quanti nodi film hanno almeno 20 collegamenti (valutazioni)
num_movies_with_more_than_20_connections = sum(1 for count in movie_out_degree_count.values() if count >= 8000)

# Stampa il numero di nodi film con almeno 20 collegamenti
print(f"Numero di nodi film con almeno 100 collegamenti: {num_movies_with_more_than_20_connections}")





Numero di nodi film con almeno 100 collegamenti: 769


In [ ]:
import networkx as nx
G_tot = nx.Graph(G)

In [ ]:

# Trova i nodi film (nella seconda bipartizione) con meno di 30 collegamenti e rimuovili
for node in G.nodes():
    if G.nodes[node]['bipartite'] == 1 and G.degree[node] < 8000:
        G_tot.remove_node(node)

# Verifica il numero di nodi film rimasti
# Verifica il numero di nodi film rimasti nella copia
num_movies_remaining = len([node for node in G_tot.nodes() if G_tot.nodes[node]['bipartite'] == 1])

# Stampa il numero di film rimasti nella copia
print(f"Numero di film con almeno 30 collegamenti nella copia (G_tot): {num_movies_remaining}")

# Stampa il numero di film rimasti
print(f"Numero di film con almeno 30 collegamenti: {num_movies_remaining}")


Numero di film con almeno 30 collegamenti nella copia (G_tot): 769
Numero di film con almeno 30 collegamenti: 769


In [ ]:
import pandas as pd
movies = pd.read_csv('movies.csv')
movies.tail()

,movieId,title,genres
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)
62422,209171,Women of Devil's Island (1962),Action|Adventure|Drama


In [ ]:
import pandas as pd

# Crea un elenco dei film rimasti nella rete G_tot
movies_remaining = [node for node in G_tot.nodes() if G_tot.nodes[node]['bipartite'] == 1]

# Filtra il DataFrame movies per mantenere solo i film rimasti
filtered_movies_df = movies[movies['movieId'].isin(movies_remaining)]

# Stampa il numero di film rimasti
num_movies_remaining = len(filtered_movies_df)
print(f"Numero di film rimasti: {num_movies_remaining}")

# Stampa il DataFrame con i film rimasti
print(filtered_movies_df)


Numero di film rimasti: 769
       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
4            5  Father of the Bride Part II (1995)   
5            6                         Heat (1995)   
...        ...                                 ...   
25063   122904                     Deadpool (2016)   
29538   134130                  The Martian (2015)   
29850   134853                   Inside Out (2015)   
37184   152081                     Zootopia (2016)   
42586   164179                      Arrival (2016)   

                                                  genres  
0            Adventure|Animation|Children|Comedy|Fantasy  
1                             Adventure|Children|Fantasy  
2                                         Comedy|Romance  
4                                                 Comedy  
5                           

In [ ]:

from scipy.stats import pearsonr
import numpy as np


# Ora calcoliamo la matrice di similarità tra i film
movie_nodes = [node for node in G_tot.nodes if G_tot.nodes[node]['bipartite'] == 1]
num_movies = len(movie_nodes)

similarity_matrix = np.zeros((num_movies, num_movies))  # Inizializza la matrice con zeri

for i in range(num_movies):
    for j in range(i + 1, num_movies):
        movie1 = movie_nodes[i]
        movie2 = movie_nodes[j]

        # Estrai i rating degli utenti comuni tra i due film
        common_users = set(G_tot.neighbors(movie1)) & set(G_tot.neighbors(movie2))

        if len(common_users) >= 20:
            ratings_movie1 = [G_tot[movie1][user]['weight'] for user in common_users]
            ratings_movie2 = [G_tot[movie2][user]['weight'] for user in common_users]

            # Calcola la similarità di Pearson tra i due film
            similarity = pearsonr(ratings_movie1, ratings_movie2)[0]
            similarity_matrix[i][j] = similarity
            similarity_matrix[j][i] = similarity
        else:
            # Se non ci sono abbastanza dati per il calcolo, assegna 0
            similarity_matrix[i][j] = 0
            similarity_matrix[j][i] = 0

# Ora similarity_matrix è una matrice di similarità tra i film.
# Puoi accedere alle similarità tra film specifici utilizzando l'indice dei film.



In [ ]:
filtered_movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller


In [ ]:
movie_nodes

[296,
 899,
 1088,
 1250,
 1653,
 2011,
 2012,
 2161,
 2692,
 3448,
 3949,
 4308,
 4973,
 5952,
 6016,
 6377,
 6539,
 6711,
 7361,
 8360,
 1,
 62,
 110,
 150,
 151,
 236,
 260,
 261,
 266,
 318,
 333,
 349,
 356,
 364,
 380,
 457,
 480,
 497,
 527,
 553,
 588,
 589,
 653,
 733,
 858,
 914,
 953,
 1035,
 1080,
 1136,
 1196,
 1197,
 1198,
 1201,
 1210,
 1246,
 1270,
 1271,
 1275,
 1287,
 1291,
 1293,
 1302,
 1356,
 1374,
 1376,
 1393,
 1485,
 1527,
 1580,
 1584,
 1610,
 1674,
 1682,
 1722,
 1907,
 1923,
 1968,
 2028,
 2081,
 2115,
 2150,
 2194,
 2268,
 2273,
 2294,
 2324,
 2355,
 2406,
 2470,
 2571,
 2617,
 2640,
 2694,
 2761,
 2762,
 2797,
 2918,
 2987,
 3114,
 3175,
 3578,
 3793,
 3916,
 3948,
 3994,
 4016,
 4022,
 4306,
 4571,
 4720,
 4874,
 4886,
 4896,
 4963,
 4993,
 4995,
 5010,
 5349,
 5418,
 5445,
 5816,
 6333,
 7090,
 7143,
 7153,
 8368,
 8636,
 8665,
 8961,
 8970,
 30749,
 33166,
 33493,
 33794,
 34162,
 35836,
 40815,
 29,
 32,
 50,
 111,
 172,
 173,
 293,
 442,
 541,
 593,
 7

In [ ]:
similarity_matrix

array([[ 0.        ,  0.11070982, -0.06685249, ...,  0.00582279,
         0.23362688,  0.21129403],
       [ 0.11070982,  0.        ,  0.25628618, ...,  0.09481373,
         0.14772793,  0.19283307],
       [-0.06685249,  0.25628618,  0.        , ...,  0.23791977,
        -0.02539753,  0.04022456],
       ...,
       [ 0.00582279,  0.09481373,  0.23791977, ...,  0.        ,
         0.0579543 ,  0.04352321],
       [ 0.23362688,  0.14772793, -0.02539753, ...,  0.0579543 ,
         0.        ,  0.30023907],
       [ 0.21129403,  0.19283307,  0.04022456, ...,  0.04352321,
         0.30023907,  0.        ]])

In [ ]:
# Supponiamo che tu voglia scoprire a quale film corrisponde l'indice 3 nella matrice di similarità
indice_desiderato = 1

# Ottieni il movieId corrispondente all'indice desiderato
movieId_corrispondente = movie_nodes[indice_desiderato]

# Ora puoi utilizzare il movieId per recuperare le informazioni del film, ad esempio il titolo
# Supponiamo che tu abbia un DataFrame df contenente le informazioni sui film
film_corrispondente = filtered_movies_df[filtered_movies_df['movieId'] == movieId_corrispondente]

# Esempio di stampa del titolo del film
titolo_del_film = film_corrispondente['title'].values[0]
print(f"Il film corrispondente all'indice {indice_desiderato} è: {titolo_del_film}")


Il film corrispondente all'indice 1 è: Toy Story (1995)


In [ ]:
filtered_movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
25063,122904,Deadpool (2016),Action|Adventure|Comedy|Sci-Fi
29538,134130,The Martian (2015),Adventure|Drama|Sci-Fi
29850,134853,Inside Out (2015),Adventure|Animation|Children|Comedy|Drama|Fantasy
37184,152081,Zootopia (2016),Action|Adventure|Animation|Children|Comedy


In [ ]:
filtered_movies_df['title'].to_csv('film_rimasti.csv', index=False)

In [ ]:
with open('df_movie_remaining.pkl', 'wb') as file:
    pickle.dump(filtered_movies_df, file)

In [ ]:
with open('list_movies_for_simililarity.pkl', 'wb') as file:
    pickle.dump(movie_nodes, file)

In [ ]:
import numpy as np

def recommend_movies(movie_title, similarity_matrix, movie_nodes, df, top_n=5):
    try:
        # Trova il movieId corrispondente al titolo del film
        film_corrispondente = filtered_movies_df[filtered_movies_df['title'] == movie_title]
        movieId_corrispondente = film_corrispondente['movieId'].values[0]

        # Trova l'indice corrispondente a questo movieId nella matrice di similarità
        film_index = movie_nodes.index(movieId_corrispondente)

        # Estrai le similarità del film corrente con tutti gli altri film
        similarities = similarity_matrix[film_index]

        # Ordina gli indici delle similarità in ordine decrescente
        sorted_indices = np.argsort(similarities)[::-1]

        # Filtra i primi top_n film più simili, escludendo il film di riferimento
        recommended_indices = [i for i in sorted_indices if i != film_index][:top_n]

        # Ottieni i titoli dei film raccomandati
        recommended_movies = [filtered_movies_df[filtered_movies_df['movieId'] == movie_nodes[i]]['title'].values[0] for i in recommended_indices]

        return recommended_movies

    except IndexError:
        return "Film non trovato"

# Esempio di utilizzo della funzione
film_raccomandati = recommend_movies("Jumanji (1995)", similarity_matrix, movie_nodes, filtered_movies_df)
print(f"I 5 film più simili a 'Titolo del Film': {film_raccomandati}")


I 5 film più simili a 'Titolo del Film': ['Casper (1995)', 'Mrs. Doubtfire (1993)', 'Hook (1991)', 'Santa Clause, The (1994)', 'Home Alone 2: Lost in New York (1992)']


In [ ]:
# Specifica il percorso del file in cui desideri salvare la matrice di similarità
pickle_filename = "similarities_movies.pkl"

# Apri il file in modalità binaria per la scrittura
with open(pickle_filename, "wb") as pickle_file:
    # Serializza e salva la matrice di similarità
    pickle.dump(similarity_matrix, pickle_file)

In [ ]:
# Specifica il percorso del file dal quale desideri caricare la matrice di similarità
#pickle_filename = "similarities.pkl"

# Apri il file in modalità binaria per la lettura
#with open(pickle_filename, "rb") as pickle_file:
    # Deserializza la matrice di similarità
    #loaded_similarity_matrix = pickle.load(pickle_file)

# Ora loaded_similarity_matrix contiene la matrice di similarità recuperata dal file